# Proxy communication

Direct communication allows to control the Rover using a UART (serial) interface. This is useful for bench tests, but unsuitable for operation, as the control station runs in a computer that is not the onboard computer.

To keep the same interface, the proxy concept is used.

~~~
VehicleIF (ProxyClient) <--- ZMQ TCP/IP ----> VehicleIF (ProxyServer) <--- UART ----> Serial
~~~

This procedure describes minimal connectivity test using the proxy communication mode.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
from rover.vehicleif import VehicleIF

## Connection

In [3]:
GROUNDSTATION_HOST = "localhost"
GROUNDSTATION_PORT = 5557
VEHICLE_HOST = "localhost"
VEHICLE_PORT = 5558

### Vehicle side

In [5]:
!ls /dev/ttyACM*

/dev/ttyACM0


In [6]:
connection_params_vehicle_proxy = {
    "mode": VehicleIF.MODE_PROXY_VEHICLE,
    
    # Vehicle side needs access to the serial device
    "port": "/dev/ttyACM0",
    "baudrate": 115200,
    
    "vehicle_port": VEHICLE_PORT,
    "groundstation_host": GROUNDSTATION_HOST,
    "groundstation_port": GROUNDSTATION_PORT
}

vehicle_proxy = VehicleIF(connection_params_vehicle_proxy,debug=False)
vehicle_proxy.capture_path = "./captures"

### Ground control station side

In [7]:
connection_params = {
    "mode": VehicleIF.MODE_PROXY_GROUND,
   
    "groundstation_port": GROUNDSTATION_PORT,
    "vehicle_host": VEHICLE_HOST,
    "vehicle_port": VEHICLE_PORT
}

vehicle = VehicleIF(connection_params,debug=False)
vehicle.capture_path = "./captures"

## Operation

In [10]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 167
   OnBoardTime: 100505
   ReceivedPackets: 0
   GeneralStatus: 0x00000000
   Debug1: 0x00000000
   Debug2: 0x00000000
Motor Telemetry
   TelemetryCycle: 167
   OnBoardTime: 100755
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 21
   Tachometer3: 11
   Tachometer4: 0
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0x00000000
IMU Telemetry
No IMU telemetry frames have been received yet.


In [11]:
# Led on
vehicle.control_led(0x00000001)

Sent: 0x 40 3C 05 00 00 00 00 01 A2 0A  (10 bytes)


In [12]:
# Led off
vehicle.control_led(0x00000000)

Sent: 0x 40 3C 05 00 00 00 00 00 93 0A  (10 bytes)


## Teardown

In [13]:
vehicle_proxy.disconnect()
vehicle.disconnect()